In [1]:
# %load boiler_plate.py
# reload modules when they change
%load_ext autoreload
%autoreload 2

# Alow plotting
%matplotlib inline

# Import modules higher in folder hierarchy
import os
import sys


def add_path(path):
    if path not in sys.path:
        sys.path.append(path)


In [2]:
add_path('..')

In [3]:
from pipeline.db import fetch, inner_join, filter_videos

# Load data
## Videos data

In [4]:
dbname='ds-wizards'
user='wizard'
host='192.95.32.117'
password='GaG23jVxZhMnQaU53r8o'

VQUERY = "select post_id, url from videos where status='ok'"

vres = fetch(host, dbname, user, password, VQUERY)
vres = [(post_id.split("_")[1], url) for post_id, url in vres]

## Tags

In [5]:
dbname='ds-content-tags'
user='ds-content-tags'
password='0fXjWl592vNf1gYvIw8w'
host='192.95.32.117'

TQUERY = "select id, tags from videos where tags is not NULL"
TAGS = "select tag_id, name, path from content_tags"

tres = fetch(host, dbname, user, password, TQUERY)

tags = { 
    tag_id: (name, path) for (tag_id, name, path) in fetch(
        host, dbname, user, password, TAGS) 
}

In [6]:
import itertools

In [7]:
from collections import Counter

In [8]:
c = Counter(itertools.chain(*[tags for (_, tags) in tres]))

In [9]:
sorted([(tags[k][0], v  )for k, v in c.items() if v > 10], key = lambda x: x[1], )

[('Bill Nye', 11),
 ('Ballet', 11),
 ('Anti-racism', 11),
 ('Cannabis Now Magazine', 11),
 ('Taco Bell', 11),
 ('Kendall Jenner', 11),
 ('Denver', 11),
 ('Science fiction movies', 11),
 ('The New York Times', 11),
 ('Pokémon Go', 11),
 ('Fire', 11),
 ('Jurassic Park (film)', 11),
 ('Yemen', 11),
 ('Dancing with the Stars', 11),
 ('Grammy Award', 11),
 ('Engineering', 11),
 ('The Walking Dead (TV series)', 11),
 ('Ivanka Trump', 11),
 ('Stop violence against women', 11),
 ('Plant', 11),
 ('War', 11),
 ('The Rock - WWE', 11),
 ('Wheelchair', 11),
 ('South Korea', 11),
 ('NFL Network', 11),
 ('Farm Animal Rights Movement (FARM)', 11),
 ('Artificial intelligence', 11),
 ('Oil', 11),
 ('Finance', 11),
 ('Deportation', 11),
 ('Nintendo', 11),
 ('Iowa', 11),
 ('Sarah Silverman', 11),
 ('Beyonce News', 11),
 ('Astrophysics', 11),
 ('Information technology', 11),
 ('Pokémon', 11),
 ('Lunch', 11),
 ('Ireland', 11),
 ('The Animal Rescue Site', 11),
 ("UFO's & Aliens", 11),
 ('Beauty salons', 11),

In [10]:
Counter([len(path) for name, path in tags.values()])

Counter({2: 6, 3: 4829, 4: 150})

In [11]:
[(name, path) for name, path in tags.values() if len(path) == 2]

[('Shopping and fashion', ['Interests', 'Shopping and fashion']),
 ('Fitness and wellness', ['Interests', 'Fitness and wellness']),
 ('Entertainment', ['Interests', 'Entertainment']),
 ('Family and relationships', ['Interests', 'Family and relationships']),
 ('Sports and outdoors', ['Interests', 'Sports and outdoors']),
 ('Technology', ['Interests', 'Technology'])]

## Join videos with tags

In [12]:
videos = inner_join(tres, vres)
filtered, t2i, i2t = filter_videos(videos, 10)
print("Found %d videos with %d unique tags" % (len(filtered), len(t2i)))

Found 9819 videos with 492 unique tags


In [41]:
len(t2i)

492

In [14]:
v = list(inner_join(tres, vres))

In [ ]:
# Take only the most common class from the list

In [28]:
import math

c = Counter([min([t2i.get(t, math.inf) for t in tags]) for (id, tags, url) in v])

In [40]:
len(c)

407